### Name - Yash Ginoya
### Roll Number - 20BCE075
### Practical 4 - Apply Transfer Learning on CIFAR-10 Dataset

In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
import sklearn
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [9]:
# Load the CIFAR10 dataset
(xtrain,ytrain), (xtest,ytest) = datasets.cifar10.load_data()

In [10]:
xtrain = tf.keras.applications.vgg19.preprocess_input(xtrain)
xtest = tf.keras.applications.vgg19.preprocess_input(xtest)

In [11]:
xtrain,xval,ytrain,yval=train_test_split(xtrain,ytrain,test_size=.3)

In [12]:
ytrain = to_categorical(ytrain)
ytest = to_categorical(ytest)
yval = to_categorical(yval)

In [13]:
print((xtrain.shape,ytrain.shape))
print((xval.shape,yval.shape))
print((xtest.shape,ytest.shape))

((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


In [14]:
train_generator = ImageDataGenerator(
    rotation_range=20, 
    horizontal_flip=True,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1)

val_generator = ImageDataGenerator(
    rotation_range=20, 
    horizontal_flip=True,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1)

test_generator = ImageDataGenerator(
    rotation_range=20, 
    horizontal_flip=True,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1)


In [15]:
train_generator.fit(xtrain)
val_generator.fit(xval)
test_generator.fit(xtest)

In [17]:
base_model1 = keras.applications.resnet50.ResNet50(include_top=False, input_shape=(32, 32, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [40]:
base_model1.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 38, 38, 3)    0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 16, 16, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 16, 16, 64)   256         ['conv1_conv[0][0]']             
                                                                                           

In [19]:
for layer in base_model1.layers:
    layer.trainable = False

model3 = models.Sequential()
model3.add(base_model1)
model3.add(layers.Flatten())
model3.add(layers.Dense(1024, activation='relu',input_dim=512))
model3.add(layers.Dense(512, activation='relu'))
model3.add(layers.Dense(256, activation='relu'))
model3.add(layers.Dense(128, activation='relu'))
model3.add(layers.Dropout(0.5))
model3.add(layers.Dense(10, activation='softmax'))

In [ ]:
# Compile the model with appropriate loss and optimizer
model3.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

# Train the model using transfer learning
history = model3.fit(xtrain,ytrain, epochs=30, validation_data=(xtest, ytest), batch_size=1024)

# Evaluate the model on the test dataset
test_loss, test_acc = model3.evaluate(xtest, ytest, verbose='auto')
print("Test accuracy:", test_acc)


Epoch 1/30
35/35 [==============================] - 15s 149ms/step - loss: 2.2395 - accuracy: 0.2476 - val_loss: 1.6648 - val_accuracy: 0.4422
Epoch 2/30
35/35 [==============================] - 3s 81ms/step - loss: 1.7353 - accuracy: 0.3880 - val_loss: 1.4496 - val_accuracy: 0.5021
Epoch 3/30
35/35 [==============================] - 3s 83ms/step - loss: 1.5691 - accuracy: 0.4540 - val_loss: 1.3577 - val_accuracy: 0.5322
Epoch 4/30
35/35 [==============================] - 3s 84ms/step - loss: 1.4734 - accuracy: 0.4867 - val_loss: 1.2914 - val_accuracy: 0.5522
Epoch 5/30
35/35 [==============================] - 3s 82ms/step - loss: 1.4152 - accuracy: 0.5133 - val_loss: 1.2725 - val_accuracy: 0.5567
Epoch 6/30
35/35 [==============================] - 3s 83ms/step - loss: 1.3615 - accuracy: 0.5269 - val_loss: 1.2226 - val_accuracy: 0.5729
Epoch 7/30
35/35 [==============================] - 3s 83ms/step - loss: 1.3176 - accuracy: 0.5428 - val_loss: 1.2097 - val_accuracy: 0.5774
Epoch 8/30
